## Grab 2017 energy data for each industry subsector and economy

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from openpyxl import Workbook
import xlsxwriter
import pandas.io.formats.excel
import glob
from pandas import ExcelWriter

In [2]:
# Load data frames

EGEDA_years = pd.read_csv('../data/input_data/EGEDA_to2017.csv')

In [3]:
# Required things

EGEDA_industry = ['13_industry_sector', '13_1_iron_and_steel', '13_2_chemical_incl__petrochemical',
                  '13_3_nonferrous_metals', '13_4_nonmetallic_mineral_products', '13_5_transportation_equipment',
                  '13_6_machinery', '13_7_mining_and_quarrying', '13_8_food_beverages_and_tobacco',
                  '13_9_pulp_paper_and_printing', '13_10_wood_and_wood_products', '13_11_construction',
                  '13_12_textiles_and_leather', '13_13_nonspecified_industry']

EGEDA_fuel = ['1_1_1_coking_coal', '1_x_coal_thermal', '1_3_lignite', '2_coal_products', '3_1_crude_oil',
              '3_x_NGLs', '4_1_1_motor_gasoline', '4_1_2_aviation_gasoline', '4_2_naphtha', '4_3_jet_fuel',
              '4_4_other_kerosene', '4_5_gas_diesel_oil', '4_6_fuel_oil', '4_7_lpg', '4_8_refinery_gas_not_liq',
              '4_9_ethane', '4_10_other_petroleum_products', '5_1_natural_gas', '9_1_fuel_wood_and_woodwaste', 
              '9_2_bagasse', '9_3_charcoal', '9_4_other_biomass', '9_5_biogas', '9_6_industrial_waste',
              '9_7_municipal_solid_waste', '9_8_1_biogasoline', '9_8_2_biodiesel', '9_8_4_other_liquid_biofuels',
              '10_electricity', '11_heat']


# Economy vector
economies = EGEDA_years['economy'].unique()

In [4]:
##################################### Build workbook with 2008 to 2017 energy use for subsectors #####################################

# Define directory where to save the workbook
script_dir = '../results/'
results_dir = os.path.join(script_dir, 'initial_look/')
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

# Create a pandas ExcelWriter workbook using xlsxwriter as the engine and save it to the directory created above

writer = pd.ExcelWriter(results_dir + 'energy_intensity_2008_2017_workbook.xlsx', engine = 'xlsxwriter')
pandas.io.formats.excel.ExcelFormatter.header_style = None

# Build workbook

for economy in economies:
    economy_df = EGEDA_years[(EGEDA_years['economy'] == economy) &
                             (EGEDA_years['fuel_code'].isin(EGEDA_fuel))].copy().reset_index(drop = True).loc[:, ['economy', 'fuel_code', 'item_code_new', '2008', '2009', '2010', 
                                                                                                                  '2011', '2012', '2013', '2014', '2015', '2016', '2017']]

    sector_dataframe = pd.DataFrame()
    for sector in EGEDA_industry:
        sector_df = economy_df[economy_df['item_code_new'] == sector].copy().reset_index(drop = True)
        sector_dataframe = sector_dataframe.append(sector_df)

    # place the dataframes within the initial look workbook
    sector_dataframe.to_excel(writer, sheet_name = economy, index = False, startrow = 0)

writer.save()

In [5]:
# Now do the same but only with 2017

##################################### Build workbook with 2017 energy use for subsectors #####################################

# Define directory where to save the workbook
script_dir = '../results/'
results_dir = os.path.join(script_dir, 'initial_look/')
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

# Create a pandas ExcelWriter workbook using xlsxwriter as the engine and save it to the directory created above

writer = pd.ExcelWriter(results_dir + 'energy_2017_workbook.xlsx', engine = 'xlsxwriter')
pandas.io.formats.excel.ExcelFormatter.header_style = None

# Build workbook

for economy in economies:
    economy_df = EGEDA_years[(EGEDA_years['economy'] == economy) &
                             (EGEDA_years['fuel_code'].isin(EGEDA_fuel)) &
                             (EGEDA_years['item_code_new'] != '13_industry_sector')].copy().reset_index(drop = True).loc[:, ['economy', 'fuel_code', 'item_code_new', '2017']]

    sector_dataframe = pd.DataFrame()
    for sector in EGEDA_industry:
        sector_df = economy_df[economy_df['item_code_new'] == sector].copy().reset_index(drop = True)
        sector_dataframe = sector_dataframe.append(sector_df)

    # place the dataframes within the initial look workbook
    sector_dataframe.to_excel(writer, sheet_name = economy, index = False, startrow = 0)

writer.save()